In [1]:
# imports
import requests
import json
import os
import sys
import platform
import time
import math
import datetime
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [2]:
# generic store data to file function
def store_data(data, file, mode='w', toJson=False):
    if toJson:
        data = json.dumps(data)
    with open(file, mode, encoding='utf-8') as fp:
        result = fp.write(data)
        return result
    
# generic load data from file function
def load_data(file, fromJson=False):
    if os.path.isfile(file):
        with open(file, 'r', encoding='utf-8', errors="ignore") as fp:
            data = fp.read()
            if fromJson:
                data = json.loads(data)
            return data
    else:
        return 'file not found'

# test text
#print(store_data('Hello', '../data/repositories/mlart/test.txt'))
#print(load_data('../data/repositories/mlart/test.txt'))

# test json
#print(store_data({'msg':'Hello World'}, '../data/repositories/mlart/test.json', toJson=True))
#print(load_data('../data/repositories/mlart/test.json', fromJson=True))

In [3]:
# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [4]:
# scan text for predefined terms

text = 'We use LSTM for anomaly and object detection. As Convolutional Neural Networks are great for ML.'

pd_ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
ml_terms = pd_ml_terms['Term'].tolist()
ml_slugs = pd_ml_terms['Slug'].tolist()
ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
ml_tags = pd_ml_terms['Tag'].tolist()
ml_tags = [x for x in ml_tags if str(x) != 'nan']

#print(ml_tags)

ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
ml_libs = ml_libs['Python Package'].tolist()

def match_text(haystack, needles, toLower = False, unique = True):
    
    if toLower == True:
        haystack = haystack.lower()
        needles = [x.lower() for x in needles]
    
    if unique == True:
        matches = {x for x in needles if x in haystack}
        matches = list(matches)
    else:
        matches = [x for x in needles if x in haystack]
    
    return matches

def match_tags(haystack):
    df = pd.read_csv('../data/patterns/ml_terms.csv')
    tags = []
    
    df.set_index('Term', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(tag)
        except:
            pass
        
    df.set_index('Slug', inplace = True)
    for item in haystack:
        try:
            tag = df.loc[item].get('Tag')
            if not 'nan' in str(tag):
                tags.append(str(tag))
        except:
            pass
        
    #if 'ANN' in tags or 'CNN' in tags or 'RNN' in tags:
    #    tags.remove('NN')
    
    return list(tags)

#ml_slugs, ml_terms, ml_libs, match_text(haystack, needles, toLower = False, unique = True)
needles = {
    'ml_slugs': ml_slugs,
    'ml_terms': ml_terms,
    'ml_libs': ml_libs,
}
needles_need_str_lower = {
    'ml_slugs': False,
    'ml_terms': True,
    'ml_libs': False,
}

matches = []

matches.extend(match_text(text, ml_terms, True))
matches.extend(match_text(text, ml_slugs, False))
print('matches', matches)

tags = match_tags(matches)
print('tags', tags)

matches ['neural network', 'detect', 'object detection', 'convolutional neural network', 'lstm', 'anomaly', 'ML']
tags ['NN', 'Object Detection', 'CNN', 'LSTM', 'ML']


In [5]:
# get file modifictaion date
# https://stackoverflow.com/questions/237079/how-to-get-file-creation-modification-date-times-in-python

def creation_date(path_to_file, datetime = True):
    """
    Try to get the date that a file was created, falling back to when it was
    last modified if that isn't possible.
    See http://stackoverflow.com/a/39501288/1709587 for explanation.
    """
    if platform.system() == 'Windows':
        timestamp = os.path.getctime(path_to_file)
    else:
        stat = os.stat(path_to_file)
        try:
            timestamp = stat.st_birthtime
        except AttributeError:
            # We're probably on Linux. No easy way to get creation dates here,
            # so we'll settle for when its content was last modified.
            timestamp = stat.st_mtime
        
    if datetime == True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

    return timestamp
    
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'
print(creation_date(folder_base+folder+notebook))

2020-12-12 16:08:03


In [6]:
# clear text formatting
def clear_text(text):
    return text.replace('\n',' ').replace('\r','').replace('¶','').strip()

In [7]:
# scrape competition

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'

def scrape_competition(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    meta['title'] = soup.find('h1', class_="competition-header__title").text.strip()
    meta['subtitle'] = soup.find('h2', class_="competition-header__subtitle").text.strip()
    meta['type'] = soup.find('p', class_="competition-header__classification-text").text.strip()
    meta['organisation'] = soup.find('span', class_="competition-header__organization-name")
    if meta['organisation'] == None:
        meta['organisation'] = ''
    else:
        meta['organisation'] = meta['organisation'].text.strip()
    temp = soup.find_all('li', class_="horizontal-list-item horizontal-list-item--bullet horizontal-list-item--default")
    for item in temp:
        #print(item.text)
        if 'team' in item.text:
            meta['teams'] = item.text.replace('teams','').replace('team','').strip()
        if 'ago' in item.text:
            #item = BeautifulSoup(item, 'html.parser')
            item = item.select('li>span>span')[0]
            meta['date_closed'] = item.get('title')
    
    meta['description'] = clear_text(soup.find('div', class_="competition-overview__content").text)
    meta['tags'] = [x.text.strip() for x in soup.find_all('span', class_="CategoryButton_CategoryName-sc-c10946 jFsDhg")]
    
    
    

    return meta

html = load_data(folder_base+folder+dataset)
if 'file not found' in html:
    print(html)
meta = scrape_competition(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'title': 'Lyft 3D Object Detection for Autonomous Vehicles', 'subtitle': 'Can you advance the state of the art in 3D object detection?', 'type': 'Featured prediction Competition', 'organisation': 'Lyft', 'teams': '547', 'date_closed': 'Wed Nov 13 2019 00:59:00 GMT+0100 (Mitteleuropäische Normalzeit)', 'description': 'Self-driving technology presents a rare opportunity to improve the quality of life in many of our communities. Avoidable collisions, single-occupant commuters, and vehicle emissions are choking cities, while infrastructure strains under rapid urban growth. Autonomous vehicles are expected to redefine transportation and unlock a myriad of societal, environmental, and economic benefits. You can apply your data analysis skills in this competition to advance the state of self-driving technology. Lyft, whose mission is to improve people’s lives with the world’s best transportation, is investing in the future of self-driving vehicles. Level 5, their self-driving division, is wo

In [8]:
# iterate competions

url = url = 'https://www.kaggle.com/c/'
folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/'
dataset = 'dataset.html'
file_out = 'meta.json'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', i, len(folders))
    #print(folder)
    #print('###', i, os.path.join(folder_base,folder,subfolder))
    path = os.path.join(folder_base,folder,dataset)
    i += 1

    if os.path.exists(path):
        print(path)
        html = load_data(path)
        if 'file not found' in html:
            print(html)
        meta = scrape_competition(html)
        meta['link'] = url+folder
        #print(meta)
        store_data(meta, os.path.join(folder_base,folder,file_out), toJson=True)
            
    if quit!=0 and i>quit:
        break

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', i, 'items')

folder: 419
subfolder: 0 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-competition-spring-2015\dataset.html
subfolder: 1 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-spring-20152\dataset.html
subfolder: 2 419
../data/repositories/kaggle/competitions/c/20-newsgroups-ciphertext-challenge\dataset.html
subfolder: 3 419
../data/repositories/kaggle/competitions/c/3d-object-detection-for-autonomous-vehicles\dataset.html
subfolder: 4 419
../data/repositories/kaggle/competitions/c/abstraction-and-reasoning-challenge\dataset.html
subfolder: 5 419
../data/repositories/kaggle/competitions/c/accelerometer-biometric-competition\dataset.html
subfolder: 6 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-big\dataset.html
subfolder: 7 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-small\dataset.html
subfolder: 8 419
../data/repositories/kaggle/competitions/c/acquire-valued-shoppers-challenge\dataset.

subfolder: 80 419
../data/repositories/kaggle/competitions/c/data-science-bowl-2018\dataset.html
subfolder: 81 419
../data/repositories/kaggle/competitions/c/data-science-bowl-2019\dataset.html
subfolder: 82 419
../data/repositories/kaggle/competitions/c/data-science-for-good-careervillage\dataset.html
subfolder: 83 419
../data/repositories/kaggle/competitions/c/data-science-for-good-city-of-los-angeles\dataset.html
subfolder: 84 419
../data/repositories/kaggle/competitions/c/data-science-london-scikit-learn\dataset.html
subfolder: 85 419
../data/repositories/kaggle/competitions/c/datasciencebowl\dataset.html
subfolder: 86 419
../data/repositories/kaggle/competitions/c/dato-native\dataset.html
subfolder: 87 419
../data/repositories/kaggle/competitions/c/decoding-the-human-brain\dataset.html
subfolder: 88 419
../data/repositories/kaggle/competitions/c/deepfake-detection-challenge\dataset.html
subfolder: 89 419
../data/repositories/kaggle/competitions/c/deloitte-churn-prediction\dataset.

subfolder: 161 419
../data/repositories/kaggle/competitions/c/histopathologic-cancer-detection\dataset.html
subfolder: 162 419
../data/repositories/kaggle/competitions/c/hivprogression\dataset.html
subfolder: 163 419
../data/repositories/kaggle/competitions/c/home-credit-default-risk\dataset.html
subfolder: 164 419
../data/repositories/kaggle/competitions/c/home-depot-product-search-relevance\dataset.html
subfolder: 165 419
../data/repositories/kaggle/competitions/c/homesite-quote-conversion\dataset.html
subfolder: 166 419
../data/repositories/kaggle/competitions/c/hospital\dataset.html
subfolder: 167 419
../data/repositories/kaggle/competitions/c/how-much-did-it-rain\dataset.html
subfolder: 168 419
../data/repositories/kaggle/competitions/c/how-much-did-it-rain-ii\dataset.html
subfolder: 169 419
../data/repositories/kaggle/competitions/c/human-protein-atlas-image-classification\dataset.html
subfolder: 170 419
../data/repositories/kaggle/competitions/c/humpback-whale-identification\dat

subfolder: 241 419
../data/repositories/kaggle/competitions/c/mastercard-data-cleansing-competition-finals\dataset.html
subfolder: 242 419
../data/repositories/kaggle/competitions/c/mdm\dataset.html
subfolder: 243 419
../data/repositories/kaggle/competitions/c/melbourne-university-seizure-prediction\dataset.html
subfolder: 244 419
../data/repositories/kaggle/competitions/c/mens-machine-learning-competition-2018\dataset.html
subfolder: 245 419
../data/repositories/kaggle/competitions/c/mens-machine-learning-competition-2019\dataset.html
subfolder: 246 419
../data/repositories/kaggle/competitions/c/mercari-price-suggestion-challenge\dataset.html
subfolder: 247 419
../data/repositories/kaggle/competitions/c/mercedes-benz-greener-manufacturing\dataset.html
subfolder: 248 419
../data/repositories/kaggle/competitions/c/MerckActivity\dataset.html
subfolder: 249 419
../data/repositories/kaggle/competitions/c/microsoft-malware-prediction\dataset.html
subfolder: 250 419
../data/repositories/kagg

subfolder: 319 419
../data/repositories/kaggle/competitions/c/risky-business\dataset.html
subfolder: 320 419
../data/repositories/kaggle/competitions/c/rossmann-store-sales\dataset.html
subfolder: 321 419
../data/repositories/kaggle/competitions/c/rsna-intracranial-hemorrhage-detection\dataset.html
subfolder: 322 419
../data/repositories/kaggle/competitions/c/rsna-pneumonia-detection-challenge\dataset.html
subfolder: 323 419
../data/repositories/kaggle/competitions/c/rsna-str-pulmonary-embolism-detection\dataset.html
subfolder: 324 419
../data/repositories/kaggle/competitions/c/RTA\dataset.html
subfolder: 325 419
../data/repositories/kaggle/competitions/c/RxVolumePrediction\dataset.html
subfolder: 326 419
../data/repositories/kaggle/competitions/c/santa-2019-revenge-of-the-accountants\dataset.html
subfolder: 327 419
../data/repositories/kaggle/competitions/c/santa-gift-matching\dataset.html
subfolder: 328 419
../data/repositories/kaggle/competitions/c/santa-workshop-tour-2019\dataset.h

subfolder: 398 419
../data/repositories/kaggle/competitions/c/whale-categorization-playground\dataset.html
subfolder: 399 419
../data/repositories/kaggle/competitions/c/whale-detection-challenge\dataset.html
subfolder: 400 419
../data/repositories/kaggle/competitions/c/WhatDoYouKnow\dataset.html
subfolder: 401 419
../data/repositories/kaggle/competitions/c/whats-cooking\dataset.html
subfolder: 402 419
../data/repositories/kaggle/competitions/c/whats-cooking-kernels-only\dataset.html
subfolder: 403 419
../data/repositories/kaggle/competitions/c/WIC2011\dataset.html
subfolder: 404 419
../data/repositories/kaggle/competitions/c/wikichallenge\dataset.html
subfolder: 405 419
../data/repositories/kaggle/competitions/c/wise-2014\dataset.html
subfolder: 406 419
../data/repositories/kaggle/competitions/c/womens-machine-learning-competition-2018\dataset.html
subfolder: 407 419
../data/repositories/kaggle/competitions/c/womens-machine-learning-competition-2019\dataset.html
subfolder: 408 419
../d

In [9]:
# collect all meta.json into a single csv

folder_base = '../data/repositories/kaggle/competitions/c/'
file_meta = 'meta.json'
fp_csv = '../data/database/kaggle_competitions.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0

runtime_start = time.time()
df = pd.DataFrame()

runtime_start = time.time()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', i, len(folders))
    #print(folder)
    #print('###', i, os.path.join(folder_base,folder,subfolder))
    path = os.path.join(folder_base,folder,file_meta)
    i += 1

    if os.path.exists(path):
        print(path)
        data = load_data(path, fromJson=True)
        if 'file not found' in data:
            print(html)
            
        data['description'] = clear_text(data['description'])

        # date (ignoring GMT+x)
        date_time_str = data['date_closed'].split('GMT')
        date_time_str = date_time_str[0].strip()
        date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')
        data['date_closed'] = date_time_obj
        
        df = df.append(data, ignore_index=True)
            
    if quit!=0 and i>quit:
        break
        
runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', i, 'items')
print(df.shape)
print(df.head())
        
# drop columns
#df.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)

folder: 419
subfolder: 0 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-competition-spring-2015\meta.json
subfolder: 1 419
../data/repositories/kaggle/competitions/c/15-071x-the-analytics-edge-spring-20152\meta.json
subfolder: 2 419
../data/repositories/kaggle/competitions/c/20-newsgroups-ciphertext-challenge\meta.json
subfolder: 3 419
../data/repositories/kaggle/competitions/c/3d-object-detection-for-autonomous-vehicles\meta.json
subfolder: 4 419
../data/repositories/kaggle/competitions/c/abstraction-and-reasoning-challenge\meta.json
subfolder: 5 419
../data/repositories/kaggle/competitions/c/accelerometer-biometric-competition\meta.json
subfolder: 6 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-big\meta.json
subfolder: 7 419
../data/repositories/kaggle/competitions/c/acm-sf-chapter-hackathon-small\meta.json
subfolder: 8 419
../data/repositories/kaggle/competitions/c/acquire-valued-shoppers-challenge\meta.json
subfolder: 9 419
../da

subfolder: 85 419
../data/repositories/kaggle/competitions/c/datasciencebowl\meta.json
subfolder: 86 419
../data/repositories/kaggle/competitions/c/dato-native\meta.json
subfolder: 87 419
../data/repositories/kaggle/competitions/c/decoding-the-human-brain\meta.json
subfolder: 88 419
../data/repositories/kaggle/competitions/c/deepfake-detection-challenge\meta.json
subfolder: 89 419
../data/repositories/kaggle/competitions/c/deloitte-churn-prediction\meta.json
subfolder: 90 419
../data/repositories/kaggle/competitions/c/deloitte-western-australia-rental-prices\meta.json
subfolder: 91 419
../data/repositories/kaggle/competitions/c/demand-forecasting-kernels-only\meta.json
subfolder: 92 419
../data/repositories/kaggle/competitions/c/denoising-dirty-documents\meta.json
subfolder: 93 419
../data/repositories/kaggle/competitions/c/detecting-insults-in-social-commentary\meta.json
subfolder: 94 419
../data/repositories/kaggle/competitions/c/diabetic-retinopathy-detection\meta.json
subfolder: 95

../data/repositories/kaggle/competitions/c/icdar2013-gender-prediction-from-handwriting\meta.json
subfolder: 172 419
../data/repositories/kaggle/competitions/c/icdar2013-stroke-recovery-from-offline-data\meta.json
subfolder: 173 419
../data/repositories/kaggle/competitions/c/icdm-2015-drawbridge-cross-device-connections\meta.json
subfolder: 174 419
../data/repositories/kaggle/competitions/c/ieee-fraud-detection\meta.json
subfolder: 175 419
../data/repositories/kaggle/competitions/c/imagenet-object-localization-challenge\meta.json
subfolder: 176 419
../data/repositories/kaggle/competitions/c/imaterialist-challenge-fashion-2018\meta.json
subfolder: 177 419
../data/repositories/kaggle/competitions/c/imaterialist-challenge-FGVC2017\meta.json
subfolder: 178 419
../data/repositories/kaggle/competitions/c/imaterialist-challenge-furniture-2018\meta.json
subfolder: 179 419
../data/repositories/kaggle/competitions/c/imaterialist-fashion-2019-FGVC6\meta.json
subfolder: 180 419
../data/repositorie

subfolder: 254 419
../data/repositories/kaggle/competitions/c/msk-redefining-cancer-treatment\meta.json
subfolder: 255 419
../data/repositories/kaggle/competitions/c/multi-modal-gesture-recognition\meta.json
subfolder: 256 419
../data/repositories/kaggle/competitions/c/multilabel-bird-species-classification-nips2013\meta.json
subfolder: 257 419
../data/repositories/kaggle/competitions/c/MusicHackathon\meta.json
subfolder: 258 419
../data/repositories/kaggle/competitions/c/new-york-city-taxi-fare-prediction\meta.json
subfolder: 259 419
../data/repositories/kaggle/competitions/c/nfl-big-data-bowl-2020\meta.json
subfolder: 260 419
../data/repositories/kaggle/competitions/c/nfl-playing-surface-analytics\meta.json
subfolder: 261 419
../data/repositories/kaggle/competitions/c/NFL-Punt-Analytics-Competition\meta.json
subfolder: 262 419
../data/repositories/kaggle/competitions/c/nips-2017-defense-against-adversarial-attack\meta.json
subfolder: 263 419
../data/repositories/kaggle/competitions/c

../data/repositories/kaggle/competitions/c/sberbank-russian-housing-market\meta.json
subfolder: 336 419
../data/repositories/kaggle/competitions/c/second-annual-data-science-bowl\meta.json
subfolder: 337 419
../data/repositories/kaggle/competitions/c/see-click-predict-fix\meta.json
subfolder: 338 419
../data/repositories/kaggle/competitions/c/seizure-detection\meta.json
subfolder: 339 419
../data/repositories/kaggle/competitions/c/seizure-prediction\meta.json
subfolder: 340 419
../data/repositories/kaggle/competitions/c/SemiSupervisedFeatureLearning\meta.json
subfolder: 341 419
../data/repositories/kaggle/competitions/c/sentiment-analysis-on-movie-reviews\meta.json
subfolder: 342 419
../data/repositories/kaggle/competitions/c/severstal-steel-defect-detection\meta.json
subfolder: 343 419
../data/repositories/kaggle/competitions/c/sf-crime\meta.json
subfolder: 344 419
../data/repositories/kaggle/competitions/c/shelter-animal-outcomes\meta.json
subfolder: 345 419
../data/repositories/kagg

subfolder: 418 419
../data/repositories/kaggle/competitions/c/zillow-prize-1\meta.json


NameError: name 'j' is not defined

In [10]:
# scrape notebook content

folder_base = '../data/repositories/kaggle/competitions/c/'
folder = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
notebook = 'notebook_02.html'

def scrape_notebook_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    meta = {}
    
    meta['author'] = soup.find('a', class_="sc-paWCZ jvKDQp").get('href').replace('/','')
    meta['title'] = soup.find('a', class_="KernelViewerContext_KernelTitle-sc-rdaqnd").text   
    meta['type'] = soup.find('span', class_="KernelViewerContext_KernelTypeInfo-sc-1l6fza6 kqxzvL").text.replace('using data from','').strip()
    meta['sources'] = soup.find('a', class_="KernelViewerContext_DataSourceUrl-sc-1dm3ij9 lpoMHV").text.strip()
    meta['tags'] = soup.find('span', class_="KernelViewerContext_CategoriesWrapper-sc-8yrjj NgcTE").text.split('·')
    meta['tags'] = list(filter(None, meta['tags']))
    meta['votes'] = soup.find('span', class_="vote-button__vote-count").text
    meta['submission'] = soup.select('div.kernel-code-pane__submission-info-content')
    if len(meta['submission']) > 0:
        meta['submission'] = meta['submission'][0].text
    #meta['votes'] = soup.find('span', class_="vote-button__vote-count").text
    meta['license'] = soup.select('div.kernel-code-pane__subtitle>a')
    if len(meta['license']) > 0:
        meta['license'] = meta['license'][0].text
    
    score = soup.select('div.kernel-code-pane__submission-score-value')
    if len(score) > 0:
        meta['score_private'] = score[0].text
        meta['score_public'] = score[1].text
    
    
    temp = soup.find('span', class_="KernelViewerContext_KernelSubtitle-sc-rltxca esPWpV").select('span') #.text.split('·')
    #print (temp)
    for item in temp:
        #print(item)
        if 'views' in item.text:
            meta['views'] = int(''.join(filter(str.isdigit, item.text)))
        if 'GMT' in item.get('title', 'nan'):
            meta['date'] = item.get('title')
    
    return meta

html = load_data(folder_base+folder+notebook)
if 'file not found' in html:
    print(html)
meta = scrape_notebook_content(html)
print(meta)
#store_data(links, folder_base+folder+out, toJson=True)

{'author': 'asimandia', 'title': 'lyft3d inference kernel', 'type': 'Python notebook', 'sources': 'multiple data sources', 'tags': ['gpu'], 'votes': '38', 'submission': 'Best Submission SuccessfulSubmitted by [ods.ai] blonde & asimandia in Kitti lands a year ago', 'license': 'Apache 2.0', 'score_private': '0.040', 'score_public': '0.040', 'views': 2595, 'date': 'Tue Oct 29 2019 16:27:40 GMT+0100 (Mitteleuropäische Normalzeit)'}


In [11]:
# scrape kernel content

folder_base = '../data/repositories/kaggle/competitions/c/'
folder1 = '3d-object-detection-for-autonomous-vehicles/notebooks/asimandia/lyft3d-inference-kernel/'
folder2 = 'airbnb-recruiting-new-user-bookings/notebooks/datadave/ndcg-score-r/'
notebook = 'notebook_02.html'
kernel = 'kernel.html'

def scrape_kernel_content(html, needles, snippet = False):
    soup = BeautifulSoup(html, 'html.parser')
    
    if snippet == True:
        code = soup.find('div', {"id": "code"}).text
    else:
        code = soup.find('body').text
    #print(code)
    
    meta = {}
    
    for key, value in needles.items():
        meta[key] = match_text(code, value, needles_need_str_lower[key])
        
    meta['ml_tags'] = match_tags(meta['ml_terms'] + meta['ml_slugs'])
        
    meta['description'] = soup.select('div.text_cell_render')
    #print(meta['description'])
    if len(meta['description']) > 0:
        meta['description'] = clear_text(meta['description'][0].text)
    else:
        meta['description'] = ''
    
    return meta

# test for non-kernel-file ('code' embedded in notebook.html)
html = load_data(folder_base+folder2+notebook)
if 'file not found' in html:
    print(html)
meta = scrape_kernel_content(html, needles)
print(meta)

# test for kernel-file ('code' in kernel.html)
html = load_data(folder_base+folder1+kernel)
if 'file not found' in html:
    print(html)
meta = scrape_kernel_content(html, needles)
print(meta)


#store_data(links, folder_base+folder+out, toJson=True)

{'ml_slugs': [], 'ml_terms': ['filter', 'train', 'classif', 'detect', 'rank', 'predict', 'deep learning', 'training data', 'fit'], 'ml_libs': [], 'ml_tags': ['Classification', 'DL'], 'description': ''}
{'ml_slugs': ['AI'], 'ml_terms': ['filter', 'train', 'epoch', 'detect', 'neural network', 'u-net', 'loss', 'convolutional neural network', 'relu', 'label', 'layer', 'predict', 'image segmentation', 'recommend', 'test data', 'fit', 'activation function', 'model'], 'ml_libs': ['pytorch'], 'ml_tags': ['NN', 'U-Net', 'CNN', 'ReLu', 'Image Segmentation', 'AI'], 'description': "Please check out Guido's excellent kernel here. In this kernel i show how to perform inference on test set using the trained model. I just added RaDAM optimzer and got some better score. You can find the BEV of the test set here. Updates: Corrected yaw calculation Used category height information"}


In [13]:
# iterate all folders and compose results into meta.json

url = 'https://www.kaggle.com/'
folder_base = '../data/repositories/kaggle/competitions/'
file_notebook = 'notebook_02.html'
file_kernel = 'kernel.html'
file_out = 'meta.json'

skip = False # if true skip meta collection / set to false to recreate {file_out} from scratch
quit = 10000 # quit after n files processed
breakOnError = False

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0
j = 0

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', len(folders))
    for subfolder in subfolders:
        print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,subfolder, 'notebooks/')
        i += 1
        
        if os.path.exists(path):
            projects = os.listdir(path)

            for author in projects:
                #print('author:', author)
                items = os.listdir(os.path.join(folder_base,folder,subfolder, 'notebooks/', author))

                for notebook in items:
                    j+=1
                    print(' - ', j, 'author:', author, '/ notebook:', notebook)
                    path_notebook = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_notebook)
                    path_kernel = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_kernel)
                    path_out = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_out)
                    
                    meta = {}
                    
                    if skip and os.path.isfile(path_out):
                        print('skipped')
                            
                    else:
                        try:
                            # get meta-data
                            if os.path.isfile(path_notebook):
                                #print('notebook found')
                                meta['scraped_at'] = creation_date(path_notebook)
                                meta['link'] = f'{url}{author}/{notebook}'
                                meta['reference'] = f'{url}{folder}/{subfolder}'
                                html = load_data(path_notebook, fromJson=False)
                                if 'file not found' in html:
                                    print('notebook not found')
                                    break
                                meta.update(scrape_notebook_content(html))

                            # get meta-data
                            if os.path.isfile(path_kernel):
                                #print('kernel found')
                                html = load_data(path_kernel, fromJson=False)
                                if 'file not found' in html:
                                    print('kernel not found')
                                    break
                                meta.update(scrape_kernel_content(html, needles))
                            else:
                                #print('kernel not found')
                                html = load_data(path_notebook, fromJson=False)
                                if 'file not found' in html:
                                    print('notebook not found')
                                    break
                                meta.update(scrape_kernel_content(html, needles))

                            #print(meta)
                            store_data(meta, path_out, toJson=True)
                            
                        except Exception as e:
                            print("Oops!", e.__class__, "occurred.")
                            print(e)
                            if breakOnError:
                                sys.exit()
                
                if j>quit:
                    break
            if j>quit:
                break
    if j>quit:
        break

folder: 1
subfolder: 1
### 0 ../data/repositories/kaggle/competitions/c\15-071x-the-analytics-edge-competition-spring-2015
### 1 ../data/repositories/kaggle/competitions/c\15-071x-the-analytics-edge-spring-20152
### 2 ../data/repositories/kaggle/competitions/c\20-newsgroups-ciphertext-challenge
 -  1 author: a45632 / notebook: classification-tfidf-svm-2-0
 -  2 author: amansohane / notebook: level-3-with-partial-deciphering-0-94-level-3
 -  3 author: ananthu017 / notebook: classification-tfidf-logistic
 -  4 author: ashishpatel26 / notebook: attension-layer-basic-for-nlp
 -  5 author: ashishpatel26 / notebook: beginner-to-intermediate-nlp-tutorial
 -  6 author: ashishpatel26 / notebook: everything-you-want-to-know-about-20-ngctc
 -  7 author: ashishpatel26 / notebook: stratified-kfold-hyperparameter-tuning
 -  8 author: delayedkarma / notebook: some-basic-explorations-lgb-baseline
 -  9 author: interneuron / notebook: difficulty-1-deciphering-wip
 -  10 author: jazivxt / notebook: enig

 -  105 author: hmendonca / notebook: u-net-model-with-submission
 -  106 author: iafoss / notebook: fine-tuning-resnet34-on-ship-detection
 -  107 author: iafoss / notebook: unet34-dice-0-87
 -  108 author: iafoss / notebook: unet34-submission-tta-0-699-new-public-lb
 -  109 author: inversion / notebook: run-length-decoding-quick-start
 -  110 author: julian3833 / notebook: 2-understanding-and-plotting-rle-bounding-boxes
 -  111 author: julian3833 / notebook: 4-exploring-public-models
 -  112 author: kmader / notebook: baseline-u-net-model-part-1
 -  113 author: kmader / notebook: from-trained-u-net-to-submission-part-2
 -  114 author: kmader / notebook: transfer-learning-for-boat-or-no-boat
 -  115 author: kotarojp / notebook: first-step-for-submission-u-net-tta
 -  116 author: leighplt / notebook: pytorch-tutorial-dataset-data-preparetion-stage
 -  117 author: meaninglesslives / notebook: airbus-ship-detection-data-visualization
 -  118 author: npatta01 / notebook: naive-model
 -  1

 -  210 author: gunesevitan / notebook: ashrae-ucf-spider-and-eda-full-test-labels
 -  211 author: hmendonca / notebook: starter-eda-and-feature-selection-ashrae3
 -  212 author: isaienkov / notebook: keras-nn-with-embeddings-for-cat-features-1-15
 -  213 author: isaienkov / notebook: lightgbm-fe-1-19
 -  214 author: jaseziv83 / notebook: a-deep-dive-eda-into-all-variables
 -  215 author: kailex / notebook: ac-dc
 -  216 author: kimtaegwan / notebook: what-s-your-cv-method
 -  217 author: nroman / notebook: eda-for-ashrae
 -  218 author: nz0722 / notebook: aligned-timestamp-lgbm-by-meter-type
 -  219 author: patrick0302 / notebook: locate-cities-according-weather-temperature
 -  220 author: purist1024 / notebook: ashrae-simple-data-cleanup-lb-1-08-no-leaks
 -  221 author: rohanrao / notebook: ashrae-half-and-half
 -  222 author: ryches / notebook: simple-lgbm-solution
 -  223 author: sudalairajkumar / notebook: simple-exploration-notebook-ashrae
 -  224 author: yamsam / notebook: ashra

 -  319 author: pradeepmuniasamy / notebook: comparative-study-of-models-geotab-inertsection
 -  320 author: pradeepmuniasamy / notebook: extensive-eda-and-modelling-geotab-inertsection
 -  321 author: prazhant / notebook: predicting-wait-times-at-intersections
 -  322 author: pulkitmehtawork1985 / notebook: beating-benchmark
 -  323 author: ragnar123 / notebook: feature-engineering-and-forward-feature-selection
 -  324 author: sanikamal / notebook: bqml-predict-wait-times
 -  325 author: sirtorry / notebook: bigquery-ml-template-intersection-congestion
 -  326 author: snugyun01 / notebook: bigqueryml-starter-code
 -  327 author: tunguz / notebook: adversarial-geotab
 -  328 author: vikassingh1996 / notebook: thoughtful-eda-feature-engineering-and-lightgbm
 -  329 author: whatust / notebook: geotab-congestion
### 35 ../data/repositories/kaggle/competitions/c\bike-sharing-demand
 -  330 author: apapiu / notebook: predicting-bike-sharing-with-xgboost
 -  331 author: benhamner / notebook:

 -  428 author: joconnor / notebook: python-xgboost-starter-0-209-public-mcc
 -  429 author: jpmiller / notebook: flowpath-viz
 -  430 author: laurae2 / notebook: what-s-in-the-kaggle-docker
 -  431 author: mcosch / notebook: leaky-36-in-r
 -  432 author: mmueller / notebook: road-2-0-4
 -  433 author: rithal / notebook: magic-feature-visualization
 -  434 author: rohanrao / notebook: r-implementation-of-mcc-optimization
 -  435 author: scirpus / notebook: loo-template-for-low-memory
### 42 ../data/repositories/kaggle/competitions/c\boston-data-festival-hackathon
### 43 ../data/repositories/kaggle/competitions/c\career-con-2019
 -  436 author: anjum48 / notebook: leakage-within-the-train-dataset
 -  437 author: artgor / notebook: basic-pytorch-lstm
 -  438 author: artgor / notebook: bayesian-optimization-for-robots
 -  439 author: artgor / notebook: where-do-the-robots-drive
 -  440 author: friedchips / notebook: the-missing-link
 -  441 author: gpreda / notebook: robots-need-help
 -  

...

 -  4503 author: pranav84 / notebook: a-beginner-s-guide-to-cern-s-trackml-challenge
 -  4504 author: shivamb / notebook: trajectory-animation-eda
 -  4505 author: sionek / notebook: bayesian-optimization
 -  4506 author: sionek / notebook: mod-dbscan-x-100-parallel
 -  4507 author: wesamelshamy / notebook: trackml-problem-explanation-and-data-exploration
 -  4508 author: yuval6967 / notebook: 7th-place-clustering-extending-ml-merging-0-75
### 376 ../data/repositories/kaggle/competitions/c\tradeshift-text-classification
 -  4509 author: tarunaryyan / notebook: imputation-for-missing-values-in-features
### 377 ../data/repositories/kaggle/competitions/c\transfer-learning-on-stack-exchange-tags
 -  4510 author: akshatpathak / notebook: text-data-clustering
 -  4511 author: anokas / notebook: frequent-words-model-v2
 -  4512 author: charlescostello / notebook: transfer-learning-on-stack-exchange-tags
 -  4513 author: eliotbarr / notebook: word-clouds
 -  4514 author: katarz / notebook: tag

 -  4608 author: enrique1500 / notebook: rental-listing-ny-map
 -  4609 author: guoday / notebook: cv-statistics-better-parameters-and-explaination
 -  4610 author: jxnlco / notebook: deduplicating-features
 -  4611 author: kashnitsky / notebook: topic-6-feature-engineering-and-feature-selection
 -  4612 author: neviadomski / notebook: data-exploration-two-sigma-renthop
 -  4613 author: poonaml / notebook: two-sigma-renthop-eda
 -  4614 author: rakhlin / notebook: another-python-version-of-it-is-lit-by-branden
 -  4615 author: robertoruiz / notebook: feature-engineering-1-sentiment-analysis
 -  4616 author: somnisight / notebook: microsoft-lightgbm-starter
 -  4617 author: stanislavushakov / notebook: python-version-of-it-is-lit-by-branden
 -  4618 author: sudalairajkumar / notebook: simple-exploration-notebook-2-connect
 -  4619 author: sudalairajkumar / notebook: xgb-starter-in-python
 -  4620 author: ygtcrt / notebook: how-to-deal-with-features-in-renthop-data
### 386 ../data/reposi

 -  4716 author: mark4h / notebook: vsb-1st-place-solution
 -  4717 author: miklgr500 / notebook: flatiron
 -  4718 author: ratthachat / notebook: demythifying-matthew-correlation-coefficients-mcc
 -  4719 author: roydatascience / notebook: eda-iso-pca-lle-stratified-lstm-attention
 -  4720 author: sohier / notebook: reading-the-data-with-python
 -  4721 author: suicaokhoailang / notebook: 5-fold-lstm-with-threshold-tuning-0-618-lb
 -  4722 author: suicaokhoailang / notebook: transformer-baseline-0-672-lb
 -  4723 author: tarunpaparaju / notebook: vsb-competition-attention-bilstm-with-features
 -  4724 author: theoviel / notebook: fast-fourier-transform-denoising
 -  4725 author: xhlulu / notebook: exploring-signal-processing-with-scipy
 -  4726 author: zoujie / notebook: analyze-power-line-signals-like-a-physicist
### 394 ../data/repositories/kaggle/competitions/c\walmart-recruiting-sales-in-stormy-weather
### 395 ../data/repositories/kaggle/competitions/c\walmart-recruiting-store-sal

 -  4821 author: rajmehra03 / notebook: a-detailed-explanation-of-keras-embedding-layer
 -  4822 author: rejasupotaro / notebook: let-s-cook-model
 -  4823 author: rejasupotaro / notebook: representations-for-ingredients
 -  4824 author: rejasupotaro / notebook: what-are-ingredients
 -  4825 author: sathyz / notebook: simple-nn-approach-kfold
 -  4826 author: shivamb / notebook: tf-idf-with-ovr-svm-what-s-cooking
 -  4827 author: tanulsingh077 / notebook: what-s-cooking
 -  4828 author: tejaeduc / notebook: whats-cooking-neural-nets-log-reg-svc
 -  4829 author: umeshnarayanappa / notebook: recipes-tf-idf-and-bigrams
 -  4830 author: wendykan / notebook: what-s-cooking-eda
### 403 ../data/repositories/kaggle/competitions/c\WIC2011
### 404 ../data/repositories/kaggle/competitions/c\wikichallenge
### 405 ../data/repositories/kaggle/competitions/c\wise-2014
### 406 ../data/repositories/kaggle/competitions/c\womens-machine-learning-competition-2018
 -  4831 author: amlanpraharaj / notebook:

 -  4928 author: xbldev / notebook: test-notebook-for-google-youtube-classification
 -  4929 author: xpeuler / notebook: frame2nparray
 -  4930 author: xpeuler / notebook: split-video-for-data-augmentation
### 416 ../data/repositories/kaggle/competitions/c\youtube8m-2018
 -  4931 author: abimannan / notebook: youtube-2m
 -  4932 author: amansrivastava / notebook: exploration-bi-lstm-model
 -  4933 author: amitkumarjaiswal / notebook: google-youtube-1st-try
 -  4934 author: anirudh257 / notebook: exploration-bi-lstm-model-by-aman-srivasthava
 -  4935 author: artyomp / notebook: a-fixed-download-script
 -  4936 author: ashishpatel26 / notebook: youtube-8m-dataset-using-extratreesclassifier
 -  4937 author: ayoubchebbi / notebook: youtube-8m-analytics
 -  4938 author: duboviy / notebook: starter-kernel-yt8m-2018-sample-data
 -  4939 author: jameschien / notebook: newbie-easy-way-for-loading-video-frame-tfrecord
 -  4940 author: juliaelliott / notebook: starter-kernel-yt8m-2018-sample-data

In [14]:
# scoring function to get a score between 0...1 for integer-values, 0.5 should be at ~100
def score(n, precision=3):
    if isinstance(n, int) or isinstance(i, float):
        return round(1-1/math.pow(1+n, 0.15), precision)
    else:
        try:
            n = int(n)
        except:
            return 0
        
    return round(1-1/math.pow(1+n, 0.15), precision)

for n in [0,1,10,25,50,100,1000,10000]:
    print(score(n))
    
print(score('3'))
print(score('a'))

0.0
0.099
0.302
0.387
0.446
0.5
0.645
0.749
0.188
0


In [15]:
# throw all parsed meta-data together in a single csv
# select only true ML cases

folder_base = '../data/repositories/kaggle/competitions/'
file_json = 'meta.json'
fp_csv = '../data/database/kaggle_competitions_01_original.csv'
fp_research = '../data/database/kaggle_competitions_02_research.csv'

quit = 0 # quit after n files processed / 0 ... no limit

folders = os.listdir(folder_base)
print('folder:', len(folders))
i = 0
j = 0

runtime_start = time.time()
df = pd.DataFrame()
df2 = pd.DataFrame()

for folder in folders:
    subfolders = os.listdir(os.path.join(folder_base,folder))
    print('subfolder:', len(folders))
    for subfolder in subfolders:
        #print('###', i, os.path.join(folder_base,folder,subfolder))
        path = os.path.join(folder_base,folder,subfolder, 'notebooks/')
        i += 1
        
        if os.path.exists(path):
            projects = os.listdir(path)

            for author in projects:
                #print('author:', author)
                items = os.listdir(os.path.join(folder_base,folder,subfolder, 'notebooks/', author))

                for notebook in items:
                    j+=1
                    #print(' - ', j, 'author:', author, '/ notebook:', notebook)
                    fp_json = os.path.join(folder_base,folder,subfolder, 'notebooks/', author, notebook, file_json)
                    
                    if os.path.isfile(fp_json):
                        data = load_data(fp_json, fromJson=True)
                        #print(data)
                        data['score_votes'] = score(data['votes'])
                        data['score_views'] = score(data['views'])
                        ml_score = 0
                        if len(data['ml_slugs']) > 0 or len(data['ml_terms']) > 0:
                            ml_score += 0.2
                        if 'ml_tags' in data and len(data['ml_tags']) > 0:
                            ml_score += 0.3
                        if len(data['ml_libs']) > 0:
                            ml_score += 0.5
                        data['ml_detected'] = ml_score
                        
                        data['description'] = data['description'].replace('\n', ' ').replace('\r', '').replace('¶', '').strip()
                        
                        # date (ignoring GMT+x)
                        # Wed Dec 19 2018 14:42:40 GMT+0100 (Mitteleuropäische Normalzeit)
                        # https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
                        date_time_str = data['date'].split('GMT')
                        date_time_str = date_time_str[0].strip()
                        date_time_obj = datetime.datetime.strptime(date_time_str, '%a %b %d %Y %H:%M:%S')

                        #print('Date:', date_time_obj.date())
                        #print('Time:', date_time_obj.time())
                        #print('Date-time:', date_time_obj)
                        data['date'] = date_time_obj
                        
                        # store only items with:
                        # - ml_score >= 0.5
                        # - are indicated as "best submission" # and 'Best Submission' in data['submission'] 
                        # - description > 1 word
                        words = data['description'].split(' ')
                        if ml_score >= 0.5 and len(words) > 5:
                            df = df.append(data, ignore_index=True)
                            print(data['link'], 'is ML use case')
                        
                        else:
                            #if 'R' in data['type']:
                            df2 = df2.append(data, ignore_index=True)
                            print(data['link'], 'is not ML use case')
                            
                    #if j % 100 == 0:
                    #    print('folder', i, '/ notebook', j)
                        
                if quit!=0 and j>quit:
                    break
            if quit!=0 and j>quit:
                break
    if quit!=0 and j>quit:
        break
        
# drop duplicates
df = df.drop_duplicates(['link'])

runtime_end = time.time()
print('runtime:', round(runtime_end - runtime_start, 3), 'seconds for', j, 'items')
print(df.shape)
print(df.head())
        
# drop columns
df.drop(columns=['author', 'submission'], inplace=True)
df2.drop(columns=['author', 'submission'], inplace=True)

df.to_csv(fp_csv, sep=';', index=False)
df2.to_csv(fp_research, sep=';', index=False)

folder: 1
subfolder: 1
https://www.kaggle.com/a45632/classification-tfidf-svm-2-0 is ML use case
https://www.kaggle.com/amansohane/level-3-with-partial-deciphering-0-94-level-3 is ML use case
https://www.kaggle.com/ananthu017/classification-tfidf-logistic is not ML use case
https://www.kaggle.com/ashishpatel26/attension-layer-basic-for-nlp is ML use case
https://www.kaggle.com/ashishpatel26/beginner-to-intermediate-nlp-tutorial is ML use case
https://www.kaggle.com/ashishpatel26/everything-you-want-to-know-about-20-ngctc is not ML use case
https://www.kaggle.com/ashishpatel26/stratified-kfold-hyperparameter-tuning is ML use case
https://www.kaggle.com/delayedkarma/some-basic-explorations-lgb-baseline is not ML use case
https://www.kaggle.com/interneuron/difficulty-1-deciphering-wip is ML use case
https://www.kaggle.com/jazivxt/enigma-layers-template is not ML use case
https://www.kaggle.com/kaggleuser58/1-char-decryption-in-level-1-2-and-3-gives-98-85 is ML use case
https://www.kaggle.

https://www.kaggle.com/hmendonca/airbus-mask-rcnn-and-coco-transfer-learning is ML use case
https://www.kaggle.com/hmendonca/classification-and-segmentation-fp is ML use case
https://www.kaggle.com/hmendonca/u-net-model-with-submission is ML use case
https://www.kaggle.com/iafoss/fine-tuning-resnet34-on-ship-detection is ML use case
https://www.kaggle.com/iafoss/unet34-dice-0-87 is not ML use case
https://www.kaggle.com/iafoss/unet34-submission-tta-0-699-new-public-lb is not ML use case
https://www.kaggle.com/inversion/run-length-decoding-quick-start is not ML use case
https://www.kaggle.com/julian3833/2-understanding-and-plotting-rle-bounding-boxes is ML use case
https://www.kaggle.com/julian3833/4-exploring-public-models is ML use case
https://www.kaggle.com/kmader/baseline-u-net-model-part-1 is ML use case
https://www.kaggle.com/kmader/from-trained-u-net-to-submission-part-2 is ML use case
https://www.kaggle.com/kmader/transfer-learning-for-boat-or-no-boat is ML use case
https://www

https://www.kaggle.com/rathimadhav/notebook79ae0de477 is not ML use case
https://www.kaggle.com/aitude/ashrae-kfold-lightgbm-without-leak-1-08 is ML use case
https://www.kaggle.com/caesarlupum/ashrae-start-here-a-gentle-introduction is ML use case
https://www.kaggle.com/corochann/ashrae-training-lgbm-by-meter-type is ML use case
https://www.kaggle.com/corochann/optuna-tutorial-for-hyperparameter-optimization is ML use case
https://www.kaggle.com/gunesevitan/ashrae-ucf-spider-and-eda-full-test-labels is ML use case
https://www.kaggle.com/hmendonca/starter-eda-and-feature-selection-ashrae3 is ML use case
https://www.kaggle.com/isaienkov/keras-nn-with-embeddings-for-cat-features-1-15 is not ML use case
https://www.kaggle.com/isaienkov/lightgbm-fe-1-19 is not ML use case
https://www.kaggle.com/jaseziv83/a-deep-dive-eda-into-all-variables is not ML use case
https://www.kaggle.com/kailex/ac-dc is not ML use case
https://www.kaggle.com/kimtaegwan/what-s-your-cv-method is ML use case
https://w

https://www.kaggle.com/danofer/baseline-feature-engineering-geotab-69-5-lb is ML use case
https://www.kaggle.com/dcaichara/feature-engineering-and-lightgbm is not ML use case
https://www.kaggle.com/fatihbilgin/data-visualization-and-eda-for-geotab-bigquery is not ML use case
https://www.kaggle.com/gaborfodor/0-feature-extraction is ML use case
https://www.kaggle.com/gaborfodor/5-combine-models is not ML use case
https://www.kaggle.com/janlauge/intersection-congestion-eda is ML use case
https://www.kaggle.com/jpmiller/eda-to-break-through-rmse-68 is not ML use case
https://www.kaggle.com/kabure/insightful-eda-modeling-lgbm-hyperopt is ML use case
https://www.kaggle.com/pradeepmuniasamy/comparative-study-of-models-geotab-inertsection is not ML use case
https://www.kaggle.com/pradeepmuniasamy/extensive-eda-and-modelling-geotab-inertsection is not ML use case
https://www.kaggle.com/prazhant/predicting-wait-times-at-intersections is ML use case
https://www.kaggle.com/pulkitmehtawork1985/bea

https://www.kaggle.com/saisatish09/a-beginner-s-guide-to-feature-selection-methods is ML use case
https://www.kaggle.com/scirpus/benouilli-naive-bayes is not ML use case
https://www.kaggle.com/timesler/xgboost-15-02-2016 is not ML use case
https://www.kaggle.com/trottefox/blending-trees is not ML use case
https://www.kaggle.com/trottefox/nearest-neighbour-linear-features is not ML use case
https://www.kaggle.com/cartographic/bish-bash-xgboost is not ML use case
https://www.kaggle.com/cartographic/de-dupe-categoricals is not ML use case
https://www.kaggle.com/cpmpml/optimizing-probabilities-for-best-mcc is ML use case
https://www.kaggle.com/danielfg/xgboost-reg-linear-lb-0-485 is not ML use case
https://www.kaggle.com/dollardollar/eda-of-important-features is not ML use case
https://www.kaggle.com/gaborfodor/69-failure-rate is ML use case
https://www.kaggle.com/gaborfodor/notebookd19d11e4f2 is ML use case
https://www.kaggle.com/gingerman/shopfloor-visualization is not ML use case
https:

...

https://www.kaggle.com/outrunner/trackml-2-solution-example is ML use case
https://www.kaggle.com/pranav84/a-beginner-s-guide-to-cern-s-trackml-challenge is not ML use case
https://www.kaggle.com/shivamb/trajectory-animation-eda is ML use case
https://www.kaggle.com/sionek/bayesian-optimization is not ML use case
https://www.kaggle.com/sionek/mod-dbscan-x-100-parallel is not ML use case
https://www.kaggle.com/wesamelshamy/trackml-problem-explanation-and-data-exploration is not ML use case
https://www.kaggle.com/yuval6967/7th-place-clustering-extending-ml-merging-0-75 is ML use case
https://www.kaggle.com/tarunaryyan/imputation-for-missing-values-in-features is not ML use case
https://www.kaggle.com/akshatpathak/text-data-clustering is ML use case
https://www.kaggle.com/anokas/frequent-words-model-v2 is not ML use case
https://www.kaggle.com/charlescostello/transfer-learning-on-stack-exchange-tags is ML use case
https://www.kaggle.com/eliotbarr/word-clouds is not ML use case
https://www

https://www.kaggle.com/enrique1500/rental-listing-ny-map is not ML use case
https://www.kaggle.com/guoday/cv-statistics-better-parameters-and-explaination is not ML use case
https://www.kaggle.com/jxnlco/deduplicating-features is not ML use case
https://www.kaggle.com/kashnitsky/topic-6-feature-engineering-and-feature-selection is ML use case
https://www.kaggle.com/neviadomski/data-exploration-two-sigma-renthop is not ML use case
https://www.kaggle.com/poonaml/two-sigma-renthop-eda is ML use case
https://www.kaggle.com/rakhlin/another-python-version-of-it-is-lit-by-branden is not ML use case
https://www.kaggle.com/robertoruiz/feature-engineering-1-sentiment-analysis is not ML use case
https://www.kaggle.com/somnisight/microsoft-lightgbm-starter is ML use case
https://www.kaggle.com/stanislavushakov/python-version-of-it-is-lit-by-branden is not ML use case
https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-2-connect is not ML use case
https://www.kaggle.com/sudalairajkum

https://www.kaggle.com/braquino/5-fold-lstm-attention-fully-commented-0-694 is ML use case
https://www.kaggle.com/braquino/vsb-power-lstm-attention is ML use case
https://www.kaggle.com/go1dfish/basic-eda is ML use case
https://www.kaggle.com/jackvial/dwt-signal-denoising is not ML use case
https://www.kaggle.com/jeffreyegan/vsb-power-line-fault-detection-approach is ML use case
https://www.kaggle.com/junkoda/handmade-features is ML use case
https://www.kaggle.com/mark4h/vsb-1st-place-solution is ML use case
https://www.kaggle.com/miklgr500/flatiron is ML use case
https://www.kaggle.com/ratthachat/demythifying-matthew-correlation-coefficients-mcc is ML use case
https://www.kaggle.com/roydatascience/eda-iso-pca-lle-stratified-lstm-attention is ML use case
https://www.kaggle.com/sohier/reading-the-data-with-python is not ML use case
https://www.kaggle.com/suicaokhoailang/5-fold-lstm-with-threshold-tuning-0-618-lb is ML use case
https://www.kaggle.com/suicaokhoailang/transformer-baseline-

https://www.kaggle.com/ashishpatel26/scrumptious-cooking-foods is ML use case
https://www.kaggle.com/ashishpatel26/think-differently-what-s-cooking is ML use case
https://www.kaggle.com/codename007/cooking-cooking-cooking is not ML use case
https://www.kaggle.com/gloriahristova/a-walkthrough-eda-vizualizations-unigram-model is not ML use case
https://www.kaggle.com/josephgpinto/cooking-is-chemistry-really is not ML use case
https://www.kaggle.com/mmotoki/word-embeddings-cuisine-similarity is ML use case
https://www.kaggle.com/nicapotato/this-model-is-bland-simple-logistic-starter is ML use case
https://www.kaggle.com/nulldata/unsupervised-cuisine-ingredients-creation is not ML use case
https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer is ML use case
https://www.kaggle.com/rejasupotaro/let-s-cook-model is ML use case
https://www.kaggle.com/rejasupotaro/representations-for-ingredients is ML use case
https://www.kaggle.com/rejasupotaro/what-are-ingredients 

https://www.kaggle.com/darrellulm/google-cloud-youtube-8m-video-testrun is not ML use case
https://www.kaggle.com/drn01z3/keras-baseline-on-video-features-0-7941-lb is not ML use case
https://www.kaggle.com/evanmiller/basic-eda is ML use case
https://www.kaggle.com/hagerrady/starter-explore-youtube8m-sample-data is ML use case
https://www.kaggle.com/kuixui/rnn-test is not ML use case
https://www.kaggle.com/mikell/most-popular-labels is not ML use case
https://www.kaggle.com/mnds18/eda-explore-youtube8m-sample-data is not ML use case
https://www.kaggle.com/neilzhang/youtube-8m is not ML use case
https://www.kaggle.com/nik0lashka/0-7941-lb-outofmemory-fix-generator-fix is not ML use case
https://www.kaggle.com/philschmidt/youtube8m-eda is ML use case
https://www.kaggle.com/pmw9440/notebook1ee341b1c9 is not ML use case
https://www.kaggle.com/veerabhadrappa/youtube8m-explorer is ML use case
https://www.kaggle.com/vlarine/most-popular-labels is not ML use case
https://www.kaggle.com/wendyka

In [16]:
# print size of gathered csv
print(df.shape)
print(df2.shape)

(2069, 21)
(2476, 21)
